In [ ]:
import os
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-service',
    'client_secret': '0edc406d-ba4a-477c-96f2-72be355d6d98',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()
WORKFLOW_RUNS = 'https://192.168.100.76:32222/api/v1/workflowruns/search?pageNo=1&pageSize=10000&sort=objectInstanceId:desc'
headers = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-468c8372-9e3c-5c3e-a115-949d07b78dfc',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-86d58331-3728-54b7-a5ae-1bd755c7878c',
    'authorization': f'Bearer {access_token}'
}
payload = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running"}
    ],
    "updatedBetween": {
        "start": "07/01/2025 00:00:00 UTC",
        "end": "07/30/2025 23:59:59 UTC"
    }
}
response = requests.post(WORKFLOW_RUNS, headers=headers, json=payload, verify=False)
workflow_ids = [item["workflowInstanceId"] for item in response.json().get("items", [])]
print(f"🔢 Total Workflows to terminate: {len(workflow_ids)}\n")
if not workflow_ids:
    print("ℹ️ No running workflow instances found. Termination skipped.")
    exit()
print("🛠️  Terminating workflows, please wait...")
spinner = ['|', '/', '-', '\\']
deleted_workflow_ids = []
spinner_index = 0
TERMINATION_URL_TEMPLATE = "https://192.168.100.76:32222/api/v1/workflowruns/{workflow_id}:terminate"
for idx, workflow_id in enumerate(workflow_ids, start=1):
    sys.stdout.write(f"\r⏳ Processing {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
    sys.stdout.flush()
    spinner_index += 1

    termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
    terminate_response = requests.post(termination_url, headers=headers, verify=False)

    if terminate_response.status_code in [200, 202]:
        deleted_workflow_ids.append(workflow_id)

    time.sleep(2)

sys.stdout.write("\r✅ All workflows terminated successfully.\n")
print(f"\n🗑️ Deleted workflow ids: {deleted_workflow_ids}")

🔢 Total Workflows to terminate: 249

🛠️  Terminating workflows, please wait...
✅ All workflows terminated successfully.

🗑️ Deleted workflow ids: [2540643, 2540245, 2537676, 2537675, 2537674, 2537670, 2537668, 2537667, 2537666, 2537665, 2537663, 2537662, 2537661, 2537660, 2537659, 2537402, 2537397, 2537394, 2537393, 2537391, 2537389, 2537386, 2537385, 2537379, 2537376, 2537375, 2537373, 2537371, 2537369, 2537364, 2537347, 2537345, 2537343, 2537341, 2537340, 2537339, 2537338, 2537337, 2537336, 2537334, 2537332, 2537330, 2537329, 2537321, 2537317, 2537309, 2537307, 2537305, 2537303, 2537301, 2537299, 2537295, 2537293, 2537292, 2537289, 2537283, 2537281, 2537279, 2537277, 2537276, 2537273, 2537272, 2537266, 2537264, 2537262, 2537259, 2537257, 2537256, 2537251, 2537249, 2537245, 2537243, 2537240, 2537239, 2537235, 2537234, 2537231, 2537230, 2537228, 2537226, 2537224, 2537221, 2537219, 2537218, 2536562, 2536559, 2536558, 2536557, 2536556, 2536554, 2536553, 2536552, 2536549, 2536548, 2536547

In [ ]:
import os
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-service',
    'client_secret': '0edc406d-ba4a-477c-96f2-72be355d6d98',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
token_response = requests.post(TOKEN_URL, data=token_data, verify=False)
access_token = token_response.json().get("access_token")
if not access_token:
    print("❌ Failed to get token.")
    exit()

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-468c8372-9e3c-5c3e-a115-949d07b78dfc',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-86d58331-3728-54b7-a5ae-1bd755c7878c',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://192.168.100.76:32222/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://192.168.100.76:32222/api/v1/workflowruns/search?pageNo=1&pageSize=100&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running"}
    ],
    "updatedBetween": {
        "start": "07/01/2025 00:00:00 UTC",
        "end": "07/30/2025 23:59:59 UTC"
    }
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False)
    workflow_ids = [item["workflowInstanceId"] for item in response.json().get("items", [])]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found. Termination complete.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
        terminate_response = requests.post(termination_url, headers=HEADERS, verify=False)

        if terminate_response.status_code in [200, 202]:
            total_deleted += 1

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


In [ ]:
import os
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-service',
    'client_secret': '0edc406d-ba4a-477c-96f2-72be355d6d98',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

try:
    token_response = requests.post(TOKEN_URL, data=token_data, verify=False, timeout=10)
    token_response.raise_for_status()
    access_token = token_response.json().get("access_token")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    sys.exit(1)

if not access_token:
    print("❌ Token not found in response.")
    sys.exit(1)

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-468c8372-9e3c-5c3e-a115-949d07b78dfc',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-86d58331-3728-54b7-a5ae-1bd755c7878c',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://192.168.100.76:32222/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://192.168.100.76:32222/api/v1/workflowruns/search?pageNo=1&pageSize=100&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running"}
    ],
    "updatedBetween": {
        "start": "07/01/2025 00:00:00 UTC",
        "end": "07/30/2025 23:59:59 UTC"
    }
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    try:
        response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json() if response.content else {}
    except Exception as e:
        print(f"\n⚠️ API request failed: {e}")
        break

    workflow_ids = [item.get("workflowInstanceId") for item in data.get("items", []) if item.get("workflowInstanceId")]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found or API returned empty response.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        try:
            termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
            terminate_response = requests.post(termination_url, headers=HEADERS, verify=False, timeout=10)

            if terminate_response.status_code in [200, 202]:
                total_deleted += 1
        except Exception as e:
            print(f"\n⚠️ Failed to terminate workflow {workflow_id}: {e}")

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


In [2]:
import os
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://qa.onprem.icedq.com/auth/realms/icedq.qa/protocol/openid-connect/token'
token_data = {
   'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu.g',
    'password': 'Admin@1234'
}

try:
    token_response = requests.post(TOKEN_URL, data=token_data, verify=False, timeout=10)
    token_response.raise_for_status()
    access_token = token_response.json().get("access_token")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    sys.exit(1)

if not access_token:
    print("❌ Token not found in response.")
    sys.exit(1)

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-6f3fdf4c-9c27-5420-a6d5-476d2549dff6',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-36655681-7ca9-58e1-b119-71ddb41cad1e',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://qa.onprem.icedq.com/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://qa.onprem.icedq.com/api/v1/workflowruns/search?pageNo=1&pageSize=100&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running"}
    ],
    "updatedBetween": {
        "start": "08/23/2025 00:00:00 UTC",
        "end": "08/24/2025 23:59:59 UTC"
    }
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    try:
        response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json() if response.content else {}
    except Exception as e:
        print(f"\n⚠️ API request failed: {e}")
        break

    workflow_ids = [item.get("workflowInstanceId") for item in data.get("items", []) if item.get("workflowInstanceId")]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found or API returned empty response.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        try:
            termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
            terminate_response = requests.post(termination_url, headers=HEADERS, verify=False, timeout=10)

            if terminate_response.status_code in [200, 202]:
                total_deleted += 1
        except Exception as e:
            print(f"\n⚠️ Failed to terminate workflow {workflow_id}: {e}")

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


🛠️  Starting batch termination of workflows...

🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 100
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 200
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 300
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 400
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 496
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 596
🔢 Found 100 running workflows to terminate...
⏳ Terminating 100/100 \
🗑️  Batch complete. Total terminated so far: 696
🔢 Found 100 running workflows to terminate...
⏳ Terminating 17/100 |
⚠️ Failed to terminate workflow 5096349: HTTPSConn

In [ ]:
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

try:
    token_response = requests.post(TOKEN_URL, data=token_data, verify=False, timeout=10)
    token_response.raise_for_status()
    access_token = token_response.json().get("access_token")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    sys.exit(1)

if not access_token:
    print("❌ Token not found in response.")
    sys.exit(1)

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-65584c43-c749-5474-86cf-a50934d7c1a0',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://192.168.100.81:32222/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://192.168.100.81:32222/api/v1/workflowruns/search?pageNo=1&pageSize=1000&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running,submitted"}
    ]
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    try:
        response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json() if response.content else {}
    except Exception as e:
        print(f"\n⚠️ API request failed: {e}")
        break

    workflow_ids = [item.get("workflowInstanceId") for item in data.get("items", []) if item.get("workflowInstanceId")]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found or API returned empty response.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        try:
            termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
            terminate_response = requests.post(termination_url, headers=HEADERS, verify=False, timeout=10)

            if terminate_response.status_code in [200, 202]:
                total_deleted += 1
        except Exception as e:
            print(f"\n⚠️ Failed to terminate workflow {workflow_id}: {e}")

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


🛠️  Starting batch termination of workflows...

🔢 Found 1000 running workflows to terminate...
⏳ Terminating 32/1000 \

KeyboardInterrupt: 

In [2]:
import os
import sys
import time
import requests
import urllib3
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://192.168.100.76:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

try:
    token_response = requests.post(TOKEN_URL, data=token_data, verify=False, timeout=10)
    token_response.raise_for_status()
    access_token = token_response.json().get("access_token")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    sys.exit(1)

if not access_token:
    print("❌ Token not found in response.")
    sys.exit(1)

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-40e74949-46b4-5570-a32d-484a334c7583',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-6c9d2689-9b9b-505c-9f30-6c3c68ee5818',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://192.168.100.76:32222/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://192.168.100.76:32222/api/v1/workflowruns/search?pageNo=1&pageSize=1000&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running,submitted"}
    ]
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    try:
        response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json() if response.content else {}
    except Exception as e:
        print(f"\n⚠️ API request failed: {e}")
        break

    workflow_ids = [item.get("workflowInstanceId") for item in data.get("items", []) if item.get("workflowInstanceId")]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found or API returned empty response.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        try:
            termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
            terminate_response = requests.post(termination_url, headers=HEADERS, verify=False, timeout=10)

            if terminate_response.status_code in [200, 202]:
                total_deleted += 1
        except Exception as e:
            print(f"\n⚠️ Failed to terminate workflow {workflow_id}: {e}")

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


🛠️  Starting batch termination of workflows...

🔢 Found 188 running workflows to terminate...
⏳ Terminating 188/188 \
🗑️  Batch complete. Total terminated so far: 188

⚠️ API request failed: 404 Client Error:  for url: https://192.168.100.76:32222/api/v1/workflowruns/search?pageNo=1&pageSize=1000&sort=objectInstanceId:desc

🎉 Finished. Total workflows terminated: 188


In [5]:
import sys
import time
import requests
import urllib3
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", message="To exit: use 'exit', 'quit', or Ctrl-D.")

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === Authentication ===
TOKEN_URL = 'https://qa.onprem.icedq.com/auth/realms/icedq.qa/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu.g',
    'password': 'Admin@12345'
}

try:
    token_response = requests.post(TOKEN_URL, data=token_data, verify=False, timeout=10)
    token_response.raise_for_status()
    access_token = token_response.json().get("access_token")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    sys.exit(1)

if not access_token:
    print("❌ Token not found in response.")
    sys.exit(1)

# === Headers & Constants ===
HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'account-id': 'acct-6f3fdf4c-9c27-5420-a6d5-476d2549dff6',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-36655681-7ca9-58e1-b119-71ddb41cad1e',
    'authorization': f'Bearer {access_token}'
}
TERMINATION_URL_TEMPLATE = "https://qa.onprem.icedq.com/api/v1/workflowruns/{workflow_id}:terminate"
SEARCH_URL = 'https://qa.onprem.icedq.com/api/v1/workflowruns/search?pageNo=1&pageSize=1000&sort=objectInstanceId:desc'
PAYLOAD_TEMPLATE = {
    "filter": [
        {"attribute": "type", "operator": "Equals", "datatype": "string", "value": "workflow"},
        {"attribute": "status", "operator": "In", "datatype": "string", "value": "running,submitted"}
    ]
}

# === Main Batch Loop ===
total_deleted = 0
spinner = ['|', '/', '-', '\\']
spinner_index = 0

print("🛠️  Starting batch termination of workflows...\n")

while True:
    try:
        response = requests.post(SEARCH_URL, headers=HEADERS, json=PAYLOAD_TEMPLATE, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json() if response.content else {}
    except Exception as e:
        print(f"\n⚠️ API request failed: {e}")
        break

    workflow_ids = [item.get("workflowInstanceId") for item in data.get("items", []) if item.get("workflowInstanceId")]

    if not workflow_ids:
        print("\n✅ No more running workflow instances found or API returned empty response.")
        break

    print(f"🔢 Found {len(workflow_ids)} running workflows to terminate...")
    for idx, workflow_id in enumerate(workflow_ids, start=1):
        sys.stdout.write(f"\r⏳ Terminating {idx}/{len(workflow_ids)} {spinner[spinner_index % len(spinner)]}")
        sys.stdout.flush()
        spinner_index += 1

        try:
            termination_url = TERMINATION_URL_TEMPLATE.format(workflow_id=workflow_id)
            terminate_response = requests.post(termination_url, headers=HEADERS, verify=False, timeout=10)

            if terminate_response.status_code in [200, 202]:
                total_deleted += 1
        except Exception as e:
            print(f"\n⚠️ Failed to terminate workflow {workflow_id}: {e}")

        time.sleep(2)  # Optional delay between requests

    print(f"\n🗑️  Batch complete. Total terminated so far: {total_deleted}")

print(f"\n🎉 Finished. Total workflows terminated: {total_deleted}")


🛠️  Starting batch termination of workflows...

🔢 Found 204 running workflows to terminate...
⏳ Terminating 204/204 \
🗑️  Batch complete. Total terminated so far: 204

⚠️ API request failed: 404 Client Error:  for url: https://qa.onprem.icedq.com/api/v1/workflowruns/search?pageNo=1&pageSize=1000&sort=objectInstanceId:desc

🎉 Finished. Total workflows terminated: 204
